# I. Import libraries
Currently, all of the medspacy subpackages are stored in separate packages/repos. We eventually want to get to havin them all as part of the `medspacy` namespace.

Some packages still need to be added to pip, which is a prerequisite for medspacy to be available as one clean install. For now, any packages not available on pip can be cloned from GitHub and installed manually (`python setup.py install`):

```bash
pip install PyRuSH
pip install cycontext
pip install clinical_sectionizer

git clone https://github.com/medspacy/nlp_preprocessor.git
git clone https://github.com/medspacy/target_matcher.git
git clone https://github.com/medspacy/nlp_postprocessor.git
```

In [1]:
import medspacy
from medspacy.visualization import visualize_ent

In [2]:
# Temporarily disable pyrush since it isn't installing on Macs
# Instead 
nlp = medspacy.load(disable=["pyrush"])

In [3]:
nlp.pipe_names

['tagger',
 'parser',
 'target_matcher',
 'sectionizer',
 'context',
 'postprocessor']

# II. Read in text
This is an excerpt from a MIMIC discharge summary.

In [4]:
with open("./discharge_summary.txt") as f:
    full_text = f.read()

In [5]:
print(full_text[:1000])

Admission Date:  [**2573-5-30**]              Discharge Date:   [**2573-7-1**]

Date of Birth:  [**2498-8-19**]             Sex:   F

Service: SURGERY

Allergies:
Hydrochlorothiazide

Attending:[**First Name3 (LF) 1893**]
Chief Complaint:
Abdominal pain

Major Surgical or Invasive Procedure:
PICC line [**6-25**]
ERCP w/ sphincterotomy [**5-31**]


History of Present Illness:
74y female with hypertension, old MI and a recent stroke affecting her
speech, who presents with 2 days of abdominal pain.  She had multiple bouts of
nausea and vomiting, with chills and decreased flatus.

Past Medical History:
1. Colon cancer dx'd in [**2554**], tx'd with hemicolectomy, XRT,
chemo. Last colonoscopy showed: Last CEA was in the 8 range
(down from 9)
2. Lymphedema from XRT, takes a diuretic
3. Cataracts
4. Hypertension
5. heart murmur - TTE in [**2567**] showed LA mod dilated, LV mildly
hypertrophied, aortic sclerosis, mild AI, mild MR.

Social History:
Married, former secretary, waitress. + tobacco 

# III. Add additional rules to various components
- **Preprocessor**: Clean up the text using the preprocessor
    - Lower-case (mainly for demo purposes)
    - Replace MIMIC-style brackets for dates and years with random dates
    - Remove all other MIMIC-style brackets
- **TargetMatcher**: Extended rule-based matching
    - Can use literal expressions or token patterns
    - Can pass in additional attributes to set for a matched attribute
    - Can also define callbacks
- **Sectionizer**: Add "Chief Complaint"
- **Postprocessor**: Modify or remove entities based on custom logic
    - As an example, we'll remove all entities  from **patient_instructions** sections

### Preprocessing

In [6]:
import re

In [7]:
import random
import time

def replace_with_random_date(match):
    return random_date("1/1/2000", "1/1/2020", random.random())

def replace_with_random_year(match):
    return str(random.randint(2000, 2020))

In [8]:
def str_time_prop(start, end, prop, format):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))


def random_date(start, end, prop, format='%m/%d/%Y'):
    return str_time_prop(start, end, prop, format)

random_date("1/1/2008", "1/1/2009", random.random())

'02/14/2008'

Unlike other components, the preprocessor isn't part of `nlp.pipeline`. Instead, it's wrapped around the tokenizer and can be accessed through `nlp.tokenizer`:

In [9]:
preprocessor = nlp.tokenizer
preprocessor

Rules for preprocessing are any callable which take the text as input and return text as output. The `PreprocessingRule` class allows for a convenient API, but you can also pass in other objects or functions.

In [10]:
from medspacy.preprocess import PreprocessingRule

In [11]:
preprocess_rules = [
    lambda x: x.lower(),
    
    PreprocessingRule(
        re.compile("\[\*\*[\d]{1,4}-[\d]{1,2}(-[\d]{1,2})?\*\*\]"),
        repl=replace_with_random_date,
        desc="Replace MIMIC date brackets with a random date."
    ),
    
    PreprocessingRule(
        re.compile("\[\*\*[\d]{4}\*\*\]"),
        repl=replace_with_random_year,
        desc="Replace MIMIC year brackets with a random year."
    ),
    
    PreprocessingRule(
        re.compile("\[\*\*[^\]]+\]"), 
        desc="Remove all other bracketed placeholder text from MIMIC"
    )
    
]

In [12]:
preprocessor.add(preprocess_rules)

In [13]:
nlp("""Admission Date:  [**2573-5-30**]              Discharge Date:   [**2573-7-1**]
 
Date of Birth:  [**2498-8-19**]
Signed electronically by: DR. [**First Name8 (NamePattern2) **]
""")

admission date:  04/04/2016              discharge date:   03/13/2014
 
date of birth:  02/05/2003
signed electronically by: dr. 

### TargetMatcher

In [14]:
from medspacy.ner import TargetRule

In [15]:
target_matcher = nlp.get_pipe("target_matcher")

In [16]:
target_rules = [
    TargetRule("abdominal pain", "PROBLEM"),
    TargetRule("hydrochlorothiazide", "TREATMENT"),
    TargetRule("cancer", "PROBLEM"),
    TargetRule("hypertension", "PROBLEM"),
    TargetRule("hypertension", "PROBLEM", pattern=[{"LOWER": "htn"}]),
    TargetRule("Lymphedema", "PROBLEM"),
    TargetRule("diuretic", "TREATMENT"),
    TargetRule("cataracts", "PROBLEM"),
    TargetRule("ulcer", "PROBLEM"),
    TargetRule("Myocardial Infarction", "PROBLEM"),
    # Give "Old MI" an explicit attribute of is_historical
    # But it still has the literal of "Myocardial Infarction"
    TargetRule("Myocardial Infarction", "PROBLEM", 
              pattern=[{"LOWER": "old"}, {"LOWER": "mi"}],
               attributes={"is_historical": True}
              ),
    TargetRule("Atrial Fibrillation", "PROBLEM", ),
]
target_matcher.add(target_rules)

In [17]:
doc = nlp("Patient with htn and old mi.")

old mi

htn



In [18]:
visualize_ent(doc)

In [19]:
print("Span", "literal", "is_historical", sep="\t\t")
for ent in doc.ents:
    print(ent, ent._.target_rule.literal, ent._.is_historical, sep="\t\t")

Span		literal		is_historical
htn		hypertension		False
old mi		Myocardial Infarction		True


### Sectionizer

In [20]:
sectionizer = nlp.get_pipe("sectionizer")

In [21]:
section_rules = [
    {"section_title": "chief_complaint", "pattern": "Chief Complaint:"}
]

In [22]:
sectionizer.add(section_rules)

### Postprocessor

In [23]:
from medspacy.postprocess import PostprocessingRule, PostprocessingPattern
from medspacy.postprocess import postprocessing_functions

In [24]:
postprocessor = nlp.get_pipe("postprocessor")

In [25]:
text = "Diagnoses: Atrial Fibrillation. Patient Instructions: Come back if you experience abdominal pain."

In [26]:
doc = nlp(text)

abdominal pain

atrial fibrillation



In [27]:
for ent in doc.ents:
    print(ent, ent._.section_title)

atrial fibrillation observation_and_plan
abdominal pain patient_instructions


In [28]:
postprocess_rules = [
    PostprocessingRule(
        patterns=[
            PostprocessingPattern(condition=lambda ent: ent._.section_title == "patient_instructions"),
        ],
        action=postprocessing_functions.remove_ent,
        description="Remove any entities from the instructions section."
    )
]

In [29]:
postprocessor.add(postprocess_rules)

In [30]:
text = "Diagnoses: Atrial Fibrillation. Patient Instructions: Come back if you experience abdominal pain."

In [31]:
doc = nlp(text)

abdominal pain
Passed: PostprocessingRule: None - Remove any entities from the instructions section. on ent: abdominal pain patient instructions: come back if you experience abdominal pain.

atrial fibrillation



In [32]:
for ent in doc.ents:
    print(ent, ent._.section_title)

atrial fibrillation observation_and_plan


In [33]:
visualize_ent(doc)

# IV. Process the entire document with the complete pipeline output

In [34]:
doc = nlp(full_text)

abdominal pain
Passed: PostprocessingRule: None - Remove any entities from the instructions section. on ent: abdominal pain please call your surgeon or return to the
emergency room if  experience fever >101.5, nausea, vomiting,
abdominal pain, shortness of breath, abdominal pain or any
significant  change in your medical condition.

abdominal pain
Passed: PostprocessingRule: None - Remove any entities from the instructions section. on ent: abdominal pain please call your surgeon or return to the
emergency room if  experience fever >101.5, nausea, vomiting,
abdominal pain, shortness of breath, abdominal pain or any
significant  change in your medical condition.

htn

atrial fibrillation

hypertension

cataracts

diuretic

lymphedema

cancer

abdominal pain

old mi

hypertension

abdominal pain

hydrochlorothiazide



In [35]:
visualize_ent(doc)